# Semantic Segmentation

We do a qualitative analysis of several pre-trained models (on [Cityscapes](https://www.cityscapes-dataset.com/) dataset) with different backbones, performing *Semantic Segmentation* on different input images.

[This notebook](https://github.com/BenoyRNair/SemanticSegmentation) was developed and tested in ***Google Colab***.

## Credits

MMSegmentation Contributors. (2020).
OpenMMLab Semantic Segmentation Toolbox and Benchmark

This runs on the [MMSegmentation](https://mmsegmentation.readthedocs.io/en/latest/) modules in [OpenMMLab](https://openmmlab.com/), and builds on:
*   [Project](https://github.com/open-mmlab/mmsegmentation)
*   [Notebook](https://github.com/open-mmlab/mmsegmentation/blob/master/demo/MMSegmentation_Tutorial.ipynb)
*   [Colab Version](https://colab.research.google.com/github/open-mmlab/mmsegmentation/blob/master/demo/MMSegmentation_Tutorial.ipynb)


## Licence
@Author [Benoy R Nair](https://github.com/BenoyRNair)

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.

You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied See the License for the specific language governing permissions and limitations under the License.




# Setup

Choose GPU (or TPU) as the 'Hardware Accelerator' for the runtime.

We install the required libraries and software in this step; it might take several minutes and might need the 'Runtime' to be restared a couple of times.

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

## Conda

*Expect to see something like:*
</br> ⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
</br> 📦 Installing...
</br> 📌 Adjusting configuration...
</br> 🩹 Patching environment...
</br> ⏲ Done in 0:00:28
</br> 🔁 Restarting kernel...


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

*Expect to see something like:*
</br> ✨🍰✨ Everything looks OK!

In [ ]:
import condacolab
condacolab.check()

## PyTorch

We use PyTorch 1.8.0 and CUDA 10.1 for this exercise.

In [ ]:
!conda install pytorch=1.8.0 torchvision cudatoolkit=10.1 -c pytorch

## MMCV

We install OpenMMLab Computer Vision Foundation Library (mmcv) in this section, and use the version that goes along with PyTorch 1.8.0.

Note: *If you are prompted to 'Restart Runtime', do so & try executing this section once again and you should see somethine like 'Requirement already satisfied...' for all the relevant packages.*

In [ ]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.8.0/index.html

## MMSegmentation

MMSegmentation repository from OpenMMLab.

Note: *If you are prompted to 'Restart Runtime', do so & try executing this section once again and you should see somethine like 'Requirement already satisfied...' for all the relevant packages.*

In [ ]:
# Clean any prior instances, if you happen to have any
!rm -rf mmsegmentation

# Clone repository and install
!git clone https://github.com/open-mmlab/mmsegmentation.git 
%cd mmsegmentation
!pip install -e .

## Checks

Confirming PyTorch and MMSegmentation installtion.

*Expect to see something like:*
</br> 1.8.0 True
</br> 0.23.0

If you see an error with mmseg, try executing the 'MMSegmentation' section again.

In [ ]:
# Check Pytorch installation
try:
  import torch, torchvision
  print(torch.__version__, torch.cuda.is_available())
except Exception as e:
  print ('Issue with torch; try executing the \'PyTorch\' section above again to install PyTorch.')
  print ("\n:-(\t" + str (e))

# Check MMSegmentation installation
try:
  import mmseg
  print(mmseg.__version__)
except Exception as e:
  print ('Issue with mmseg; try executing the \'MMSegmentation\' section above again to install MMSegmentation.')
  print ("\n:-(\t" + str (e))

# Build Model

We build the model using pre-trained weights from OpenMMLab for MMSegmentation.

## Model Configuration

We download a JSON file with links to the pre-trained model weights and the config files required for our MMSegmentation exercise. We parse the file and build the list of models, and retrieve the links to its associated model and config files.

In [ ]:
%cd /content/mmsegmentation

!pip install wget

!wget https://raw.githubusercontent.com/BenoyRNair/SemanticSegmentation/main/MMSegmentation_Model_Config.json

import json

with open ('MMSegmentation_Model_Config.json') as config_file:
    config_data = config_file.read()

config_js = json.loads (config_data)
models_list = config_js.keys()
backbones_list = (config_js[list(config_js.keys())[0]]).keys()

## Model Selection

We choose the model we want to evaluate in this section; you can choose one from the list available.

In [ ]:
#@markdown Specify the model.

from ipywidgets import interactive
import ipywidgets as widgets

def model(MODEL):
  return MODEL

model_widget = interactive (model, MODEL=models_list)
display (model_widget)

Note: The list of backbones depend on the selected model. Execute the section below to update the list of backbones for the chosen model.

In [ ]:
#@markdown Choose the backbone for the specified model.

backbones_list = (config_js[model_widget.result].keys())

def backbone(BACKBONE):
  return BACKBONE

backbone_widget = interactive (backbone, BACKBONE=backbones_list)
display (backbone_widget)

## Initialize Segmentor

We first identify the model, config & checkpoint files based on the user specified model and backbone.

We then download the pretrained weights of the model, and initialize the segmentor with the config file and checkpoint file (links to which were in the JSON file).

In [ ]:
try:
  selection = (config_js[model_widget.result])[backbone_widget.result]

  current_model = model_widget.result
  current_backbone = backbone_widget.result

  selection_model = selection ['model']
  selection_config_file = selection ['config_file']
  selection_checkpoint_file = selection ['checkpoint_file']

  #print (f'selection_model = {selection_model}')
  #print (f'selection_config_file = {selection_config_file}')
  #print (f'selection_checkpoint_file = {selection_checkpoint_file}')

  %pwd
  import wget
  import os.path

  config_folder = '/content/mmsegmentation/configs/'
  checkpoints_folder = '/content/mmsegmentation/checkpoints/'

  if not os.path.isdir (checkpoints_folder):
    os.mkdir (checkpoints_folder) 

  if not os.path.exists (checkpoints_folder + selection_checkpoint_file):
    wget.download (selection_model, checkpoints_folder)

  from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
  from mmseg.core.evaluation import get_palette

  # build the model from a config file and a checkpoint file
  model = init_segmentor (config_folder + selection_config_file, checkpoints_folder + selection_checkpoint_file, device='cuda:0')
except Exception as e:
  print ('It appears that you have selected a wrong backbone, that isn\'t available for the chosen model.')
  print ('Execute the code section above to refresh the list of backbones.')
  print ("\n:-(\t" + str (e))

# Test Model

You have an option to choose from one of the images available, or provide one by specifying the link to the image.

In [ ]:
#@markdown Specify whether you like to provide a custom image or not.

def checkbox_interactive (CUSTOM_IMAGE):
    return CUSTOM_IMAGE

cb_custom_image = interactive (checkbox_interactive, CUSTOM_IMAGE=False)
display (cb_custom_image)

In [ ]:
#@markdown Specify the input image for the segmentation task.

def get_image (IMAGE):
  return IMAGE

if not cb_custom_image.result:
  standard_images_list = [
    ('Image 1', './demo/demo.png'),
    ('Image 2', 'https://securityamp.com/assets/static/car-dash-view.b137467.dfc408b45abbc5aea697e96dfd512d05.jpg'),
    ('Image 3', 'https://upload.wikimedia.org/wikipedia/commons/thumb/e/e4/Cars_in_traffic_in_Auckland%2C_New_Zealand_-_copyright-free_photo_released_to_public_domain.jpg/1200px-Cars_in_traffic_in_Auckland%2C_New_Zealand_-_copyright-free_photo_released_to_public_domain.jpg'),
    ('Image 4', 'https://i.ibb.co/pxTxmMJ/highway-jam-road-roadway-road-marking-mark.jpg'),
    ('Image 5', 'https://upload.wikimedia.org/wikipedia/commons/e/ec/Trailer_on_the_a_busy_road.jpg'),
    ('Image 6', 'https://i.ibb.co/kDh7qfF/traffic-road-highway-way-busy-cars.jpg')
  ]

  image = interactive (get_image, IMAGE=standard_images_list)
  display (image)
else:
  image = interactive (get_image, IMAGE='Link to the image')
  display (image)

Let us now test the model with the specified image...

In [ ]:
if ((current_model != model_widget.result) or (current_backbone != backbone_widget.result)):
  print ('Model and/ or backbone has been changed.\nRun the \'Initialize Segmentor\' section again.')
else:
  try:
    img = image.result
    result = inference_segmentor(model, img)

  # show the results
    show_result_pyplot(model, img, result, get_palette('cityscapes'), title = 'Model: ' + current_model + '/ Backbone: ' + current_backbone)
  except Exception as e:
    print ('Unable to process; try another file (or choose from the list of images available, instead of the CUSTOM_IMAGE option).')
    print ("\n:-(\t" + str (e))